In [99]:
import numpy as np
import pandas as pd
import os


In [100]:
def calculate_trend_slope(data, window_size):
    if len(data) < window_size:
        raise ValueError("데이터 길이가 구간 크기보다 작습니다.")
    # float으로 data를 우선 만든다 그 이유는 소수점으로 나올 듯 하기 때문  좀더 정확하게 계산 하기 위해서
    window_data = data.astype(float)
    # x는 window size 만큼의 배열을 생성 
    # 이렇게 생성된 배열 X는 추세 선을 적합하기 위해 사용될 데이터 포인트의 인덱스를 나타낸다
    # 추세 선의 기울기를 결정하는데 사용됩니다.
    X = np.arange(window_size)
    # 함수는 다항식을 피팅하기 위해 주어진 데이터에 대해 최소 제곱 방법을 사용하여 다항식의 계수를 계산합니다.
    coefficients = np.polyfit(X, window_data, 1)  
    slope = coefficients[0]  # 추세선의 기울기를 추출합니다.
    nan_indices = np.isnan(window_data)
    # 데이터의 절대값의 평균을 계산합니다.
    abs_mean = np.mean(np.abs(window_data)) 
    # abs_mean이 0인 경우 예외를 발생시킵니다.
    if np.any(nan_indices):
        window_data = window_data[~nan_indices]
        X = X[~nan_indices]
        coefficients = np.polyfit(X, window_data, 1)
        slope = coefficients[0]
    abs_mean = np.mean(np.abs(window_data))
    
    if abs_mean == 0:
        return None
    else:
        standardized_slope = slope / abs_mean
        return standardized_slope

def calculate_trend_slope_daily(data, window_size):
    result = {}
    # 여기가 매우 중요한 부분 우리가 원하는 column을 tolist로 생성해야 한다
    column_names = data.columns.tolist()
    # 생성후 차례차례 column으로 넘겨서 각 column마다 돌아서 하기 위해 2중 for문 사용
    for column in column_names:
        # 계속해서 list를 생성할 수 없으니 for문 column마다 초기화
        trend_slope_result = []
        # 우리가 원하는 값이 data의 값과 window size값의 차이만큼을 원하므로 이런식으로 사용
        for i in range(len(data) - window_size + 1):
            #window_data에 iloc을 사용해서 i 부터 i+ window_size 만큼 준다 그 이후 window_data의 값을 선형회기 함수에 
            # 이 값을 result에 append 하고 result [column]에 넣고 처음으로 다시 돌아간다
            window_data = data[column].iloc[i:i+window_size]
            trend_slope = calculate_trend_slope(window_data, window_size)
            trend_slope_result.append(trend_slope)
        result[column] = trend_slope_result
    return result

In [101]:
# 데이터 불러오기 우리가 원하는 값들을 불러오면됨
raw = r'C:\Users\siim2\sic_project_final\data\auto_raw_3.xlsx'

df = pd.read_excel(raw, sheet_name='Sheet1', index_col=0)
df.index = df.index.strftime('%Y-%m-%d')

In [102]:
df.drop(columns=df.columns[1], inplace=True)


In [103]:
df.head()

,forward_stage,원/달러 환율,원/엔 환율,국채 3년물,국채 10년물,PER,PBR,VKOSPI,treasury_10_3_diff,T10Y2Y,...,us_e_infla,yield_gap(%),cpi,export,WTI,DGS2,DGS10,TIPS,VIX,PPI
base_date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,neutral,1154.8,1243.73,4.44,5.42,23.87,1.35,14.98,0.99,2.76,...,2.38,-0.002506,0.028,-0.086,81.52,1.09,3.85,1.47,20.04,170.8
2010-01-05,down,1140.5,1243.59,4.40,5.36,23.80,1.35,15.35,0.99,2.76,...,2.34,-0.001983,0.028,-0.086,81.74,1.01,3.77,1.43,19.35,170.8
2010-01-06,neutral,1136.4,1234.81,4.44,5.38,24.01,1.36,14.96,0.95,2.84,...,2.37,-0.002751,0.028,-0.086,83.12,1.01,3.85,1.48,19.16,170.8
2010-01-07,up,1135.4,1231.72,4.40,5.30,23.71,1.34,14.38,0.97,2.82,...,2.41,-0.001824,0.028,-0.086,82.60,1.03,3.85,1.44,19.06,170.8
2010-01-08,up,1130.5,1211.42,4.32,5.33,23.88,1.35,14.72,1.00,2.87,...,2.42,-0.001324,0.028,-0.086,82.74,0.96,3.83,1.41,18.13,170.8


In [104]:
df.info

<bound method DataFrame.info of            forward_stage  원/달러 환율   원/엔 환율  국채 3년물   국채 10년물     PER   PBR  \
base_date                                                                    
2010-01-04       neutral   1154.8  1243.73    4.440     5.420  23.87  1.35   
2010-01-05          down   1140.5  1243.59    4.400     5.360  23.80  1.35   
2010-01-06       neutral   1136.4  1234.81    4.440     5.380  24.01  1.36   
2010-01-07            up   1135.4  1231.72    4.400     5.300  23.71  1.34   
2010-01-08            up   1130.5  1211.42    4.320     5.330  23.88  1.35   
...                  ...      ...      ...      ...       ...    ...   ...   
2024-01-05            up   1315.4   906.14    3.284     3.400  18.88  0.93   
2024-01-08            up   1316.0   912.78    3.298     3.435  18.81  0.93   
2024-01-09            up   1315.7   914.16    3.256     3.348  18.76  0.93   
2024-01-10            up   1320.1   911.39    3.269     3.346  18.62  0.92   
2024-01-11            up   1312.

In [105]:
window_size_20=20

result_20 = calculate_trend_slope_daily(df.drop(columns=[ 'forward_stage']), window_size_20)

new_df_20 = df.iloc[window_size_20-1:]
result_df_20 = pd.DataFrame(result_20)


result_df_20.index=new_df_20.index

print(result_df_20)


             원/달러 환율    원/엔 환율   국채 3년물   국채 10년물        PER       PBR  \
base_date                                                                
2010-02-01  0.001400  0.003230 -0.001601 -0.000287 -0.002521 -0.002230   
2010-02-02  0.001827  0.003421 -0.001116 -0.000101 -0.002891 -0.002834   
2010-02-03  0.001912  0.003471 -0.000753 -0.000044 -0.003060 -0.003134   
2010-02-04  0.001953  0.003340 -0.000163  0.000103 -0.003009 -0.003250   
2010-02-05  0.002196  0.003586  0.000154 -0.000115 -0.003513 -0.003960   
...              ...       ...       ...       ...       ...       ...   
2024-01-05 -0.000537  0.000699 -0.004603  0.002910  0.000524  0.002047   
2024-01-08 -0.000326  0.000456 -0.004009  0.003277  0.000059  0.001610   
2024-01-09  0.000025  0.000434 -0.003249  0.003202 -0.000461  0.001142   
2024-01-10  0.000212  0.000384 -0.002475  0.002769 -0.000930  0.000659   
2024-01-11  0.000422  0.000133 -0.001785  0.002131 -0.001340  0.000177   

              VKOSPI  treasury_10_3_d

In [106]:
window_size_60=60
result_60 = calculate_trend_slope_daily(df.drop(columns=[ 'forward_stage']), window_size_60)
new_df_60 = df.iloc[window_size_60-1:]
result_df_60 = pd.DataFrame(result_60)
result_df_60.index=new_df_60.index
print(result_df_60)


             원/달러 환율    원/엔 환율   국채 3년물   국채 10년물        PER       PBR  \
base_date                                                                
2010-03-31 -0.000086  0.000014 -0.002272 -0.002108  0.000313  0.000027   
2010-04-01 -0.000094 -0.000084 -0.002242 -0.002110  0.000397  0.000144   
2010-04-05 -0.000127 -0.000200 -0.002244 -0.002116  0.000482  0.000259   
2010-04-06 -0.000166 -0.000318 -0.002220 -0.002115  0.000582  0.000386   
2010-04-07 -0.000209 -0.000444 -0.002214 -0.002154  0.000661  0.000498   
...              ...       ...       ...       ...       ...       ...   
2024-01-05 -0.000775  0.000258 -0.004593 -0.004961  0.001427  0.001717   
2024-01-08 -0.000739  0.000315 -0.004624 -0.004755  0.001405  0.001710   
2024-01-09 -0.000703  0.000375 -0.004674 -0.004563  0.001373  0.001700   
2024-01-10 -0.000674  0.000415 -0.004720 -0.004343  0.001360  0.001706   
2024-01-11 -0.000654  0.000436 -0.004789 -0.004106  0.001364  0.001729   

              VKOSPI  treasury_10_3_d

In [107]:
merge_inner = pd.merge(df, result_df_20.add_suffix('_20'), left_index=True, right_index=True)
merge_inner = pd.merge(merge_inner, result_df_60.add_suffix('_60'), left_index=True, right_index=True)


print(merge_inner)

           forward_stage  원/달러 환율   원/엔 환율  국채 3년물   국채 10년물     PER   PBR  \
base_date                                                                    
2010-03-31            up   1131.3  1213.45    3.895     4.940  24.35  1.36   
2010-04-01            up   1126.4  1204.45    3.895     4.920  24.13  1.38   
2010-04-05            up   1123.1  1188.97    3.840     4.950  24.22  1.38   
2010-04-06            up   1123.2  1195.40    3.845     4.940  24.24  1.38   
2010-04-07            up   1120.5  1191.13    3.820     4.880  24.26  1.39   
...                  ...      ...      ...      ...       ...    ...   ...   
2024-01-05            up   1315.4   906.14    3.284     3.400  18.88  0.93   
2024-01-08            up   1316.0   912.78    3.298     3.435  18.81  0.93   
2024-01-09            up   1315.7   914.16    3.256     3.348  18.76  0.93   
2024-01-10            up   1320.1   911.39    3.269     3.346  18.62  0.92   
2024-01-11            up   1312.9   902.68    3.227     3.339  1

In [108]:
base_filename = r'C:\Users\siim2\sic_project_final\data\auto_raw_20_60_{}.xlsx'

# 해당 디렉토리에 이미 존재하는 파일들을 확인
existing_files = os.listdir(r'C:\Users\siim2\sic_project_final\data')
if existing_files is True:
    print("already")
else:
    print("none")

# 가장 최신 버전 찾기
latest_version = 0
for filename in existing_files:
    if filename.startswith('result_version_') and filename.endswith('.xlsx'):
        version_number = int(filename.split('_')[2].split('.')[0])
        if version_number > latest_version:
            latest_version = version_number


none


In [110]:
# 새로운 파일명 생성
new_version = latest_version + 1
new_filename = base_filename.format(new_version)

# 파일 저장
merge_inner.to_excel(new_filename, index=True)